In [3]:

import requests
import pandas as pd

In [4]:
#downloading page
url  = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
if page.status_code == 200:
    print('Page download successful')
else:
    print('Page download error. Error code: {}'.format(page.status_code))

Page download successful


In [5]:
#Please note, I am using the pandas function "read_html" we can easily process the HTML string instead of beautifulsoup
#We set header = 0 to use the first row as column names
#Since we will discard the "Not Assigned" columns, we set them to NaN so we can later use the dropna method.
df_html = pd.read_html(url)[0]



In [11]:
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []

for key, value in df_html.iteritems(): 
    #print('value', value)
    #print ('key', key)
    
    for i in range(len(value)):
        temp=value[i]
        if 'assigned' in temp:
            continue
        #print (temp)
        pc=temp[0:3]
        #print (pc)
        neiStartId=temp.find('(')
        neiEndId=temp.find(')')
        br=temp[3:neiStartId]
        #print (br)
        stNei=temp[neiStartId+1:neiEndId]
        #print (stNei)
        #lsNei=stNei.split('/')
        #print (lsNei)

        Postcode_u.append(pc)
        Borough_u.append(br)
        nei=stNei.replace(' ','')
        nei=nei.replace('/',',')
        Neighbourhood_u.append(nei)
            #print (pc, br, nei)
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto_part1.csv')
df_toronto.shape

(103, 3)

In [12]:
df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"RougeHill,PortUnion,HighlandCreek"
2,M1E,Scarborough,"Guildwood,Morningside,WestHill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [13]:
df_toronto.loc[df_toronto['Postcode']=='M6A']

,Postcode,Borough,Neighbourhood
71,M6A,North York,"LawrenceManor,LawrenceHeights"


In [17]:

#Read CSV file from link and load into dataframe
url_csv = 'http://cocl.us/Geospatial_data'
df_coordinates = pd.read_csv(url_csv)
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
# Make sure both dataframes have the same 
df_coordinates.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df_toronto.rename(columns={'Postcode': 'PostalCode'}, inplace=True)

In [20]:
# Merge both datasets
df_neighborhoods_coordinates = pd.merge(df_toronto, df_coordinates, on='PostalCode')
df_neighborhoods_coordinates.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"RougeHill,PortUnion,HighlandCreek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,WestHill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [21]:
# Check coordinates for a couple of neighborhoods
df_neighborhoods_coordinates[(df_neighborhoods_coordinates['PostalCode']=='M5G') |
                             (df_neighborhoods_coordinates['PostalCode']=='M2H') ]

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
17,M2H,North York,HillcrestVillage,43.803762,-79.363452
57,M5G,Downtown Toronto,CentralBayStreet,43.657952,-79.387383


In [22]:
#Export to .CSV
df_neighborhoods_coordinates.to_csv('Toronto_Postcodes_2.csv')